In [1]:
# https://dplyr.tidyverse.org/reference/context.html
from datar.all import *

%run nb_helpers.py
nb_header(cur_group_id, cur_group_rows, cur_data, cur_data_all, cur_column, book='context')

[2021-06-21 13:37:44][datar][WARNING] Builtin name "min" has been overriden by datar.
[2021-06-21 13:37:44][datar][WARNING] Builtin name "max" has been overriden by datar.
[2021-06-21 13:37:44][datar][WARNING] Builtin name "sum" has been overriden by datar.
[2021-06-21 13:37:44][datar][WARNING] Builtin name "abs" has been overriden by datar.
[2021-06-21 13:37:44][datar][WARNING] Builtin name "round" has been overriden by datar.
[2021-06-21 13:37:44][datar][WARNING] Builtin name "all" has been overriden by datar.
[2021-06-21 13:37:44][datar][WARNING] Builtin name "any" has been overriden by datar.
[2021-06-21 13:37:44][datar][WARNING] Builtin name "re" has been overriden by datar.
[2021-06-21 13:37:44][datar][WARNING] Builtin name "filter" has been overriden by datar.
[2021-06-21 13:37:44][datar][WARNING] Builtin name "slice" has been overriden by datar.


### # cur_group_id  

##### gives a unique numeric identifier for the current group.


### # cur_group_rows  

##### Gives the row indices for the current group.

##### Args:
&emsp;&emsp;`_data`: The dataFrame.  

##### Returns:
&emsp;&emsp;The `_rows` from group data or row indexes (always 0-based).  


...

##### gives the current data for the current group
(excluding grouping variables).  


### # cur_data_all  

##### gives the current data for the current group
(including grouping variables)  


### # cur_column  

##### Used in the functions of across. So we don't have to register it.


In [2]:
df = tibble(
  g=['a'] + ['b'] * 2 + ['c'] * 3,
  x=runif(6),
  y=runif(6)
)
gf = df >> group_by(f.g)

gf >> summarise(n = n())

,g,n
,<object>,<int64>
0,a,1
1,b,2
2,c,3


In [3]:
gf

,g,x,y
,<object>,<float64>,<float64>
0,a,0.361852,0.652752
1,b,0.843412,0.771309
2,b,0.129364,0.244483
3,c,0.213694,0.555308
4,c,0.543648,0.825159
5,c,0.341916,0.860476


In [4]:
gf >> mutate(id=cur_group_id()) 

,g,x,y,id
,<object>,<float64>,<float64>,<int64>
0,a,0.361852,0.652752,0
1,b,0.843412,0.771309,1
2,b,0.129364,0.244483,1
3,c,0.213694,0.555308,2
4,c,0.543648,0.825159,2
5,c,0.341916,0.860476,2


In [5]:
gf >> summarise(row=cur_group_rows()) 

[2021-06-21 13:37:49][datar][   INFO] `summarise()` has grouped output by ['g'] (override with `_groups` argument)


,g,row
,<object>,<int64>
0,a,0
1,b,1
2,b,2
3,c,3
4,c,4
5,c,5


In [6]:
gf_group = gf >> summarise(data=[cur_group()])
gf_group 

,g,data
,<object>,<object>
0,a,<DF 1x1>
1,b,<DF 1x1>
2,c,<DF 1x1>


In [7]:
gf_group >> pull(f.data, to='list')

[         g
   <object>
 0        a,
          g
   <object>
 1        b,
          g
   <object>
 2        c]

In [8]:
gf_data = gf >> summarise(data=[cur_data()])
gf_data

,g,data
,<object>,<object>
0,a,<DF 1x2>
1,b,<DF 2x2>
2,c,<DF 3x2>


In [9]:
gf_data >> pull(f.data, to='list')

[          x         y
   <float64> <float64>
 0  0.361852  0.652752,
           x         y
   <float64> <float64>
 0  0.843412  0.771309
 1  0.129364  0.244483,
           x         y
   <float64> <float64>
 0  0.213694  0.555308
 1  0.543648  0.825159
 2  0.341916  0.860476]

In [10]:
gf_data_all = gf >> summarise(data=[cur_data_all()])
gf_data_all

,g,data
,<object>,<object>
0,a,<DF 1x3>
1,b,<DF 2x3>
2,c,<DF 3x3>


In [11]:
gf_data_all >> pull(f.data, to='list')

[         g         x         y
   <object> <float64> <float64>
 0        a  0.361852  0.652752,
          g         x         y
   <object> <float64> <float64>
 0        b  0.843412  0.771309
 1        b  0.129364  0.244483,
          g         x         y
   <object> <float64> <float64>
 0        c  0.213694  0.555308
 1        c  0.543648  0.825159
 2        c  0.341916  0.860476]

In [12]:
df >> select(f.x, f.y) >> mutate(
    across(
        everything(), 
        lambda x, cc: [cc + ' '] * x.shape[0] + (x**2).astype(str), cc=cur_column()
    )
)

,x,y
,<object>,<object>
0,x 0.13093660117636458,y 0.426084563003939
1,x 0.7113438529995928,y 0.5949178998177677
2,x 0.016735004359095654,y 0.05977206911640645
3,x 0.045665105202560646,y 0.30836666071983937
4,x 0.2955535761997283,y 0.6808871702451411
5,x 0.11690667798722013,y 0.7404186230214937


In [13]:
# or you can use x.name, since x is a Series
df >> mutate(across(
    [f.x, f.y], 
    lambda x: [x.name + ' '] * x.shape[0] + (x**2).astype(str)
))

,g,x,y
,<object>,<object>,<object>
0,a,x 0.13093660117636458,y 0.426084563003939
1,b,x 0.7113438529995928,y 0.5949178998177677
2,b,x 0.016735004359095654,y 0.05977206911640645
3,c,x 0.045665105202560646,y 0.30836666071983937
4,c,x 0.2955535761997283,y 0.6808871702451411
5,c,x 0.11690667798722013,y 0.7404186230214937
